In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import subprocess
import re

motif_dir = '/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM'

1)  How different are motifs present in low vs mid vs high scoring bins?
2)  How different are the motifs present within the same cluster (eg, two different motifs within cluster 1955 of the high-scoring bin)?  There may be only one motif per cluster though, we'll have to see.
3)  How different are motifs present in one cluster vs the others in the same scoring bin (eg, cluster 1955 vs the other clusters in the high-scoring bin)?
4)  How different are the motifs present in each cluster in the high-scoring bin vs each cluster in the low-scoring bin?
5)  How different are the motifs present in a given scoring bin vs each cluster in that bin (eg, the motifs in the high-scoring bin vs the motifs in cluster 1955 & other clusters in the high-scoring bin)?

In [2]:
folders = os.listdir(motif_dir)
print(folders)
f = 'cluster_H'
print(f)
df = pd.read_csv(os.path.join(motif_dir, f, 'sequences.tsv'), sep='\t')
df = df[df.motif_ID!='#']
df['motif'] = [x.split('-')[1] for x in df.motif_ID.values]
df['motif_ID'] = [int(x.split('-')[0]) for x in df.motif_ID.values]

dg = df.groupby('motif').agg({
    'seq_Score':'max'
}).reset_index()

df = df.merge(dg, on=['motif', 'seq_Score'], how='inner')
df_seq = pd.DataFrame(df[['motif_ID', 'motif', 'seq_ID', 'seq_Score', 'motif_P-value']].drop_duplicates().reset_index(drop=True))
del [df]
display(df_seq.head())
print(df_seq.shape)

['cluster_M_6', 'cluster_H', 'cluster_M_1', 'cluster_M_7', 'cluster_M_2', 'cluster_M_5', 'cluster_L', 'cluster_M_4', 'cluster_M_3']
cluster_H


,motif_ID,motif,seq_ID,seq_Score,motif_P-value
0,1,AAAACGGTTAGAC,g1|H1945|5705031,21.72,0.12
1,2,AGCGAACAGCACT,g1|H18735|4759741,21.54,0.31
2,2,AGCGAACAGCACT,g1|H11251|1008124,21.54,0.31
3,2,AGCGAACAGCACT,g1|H6514|2395052,21.54,0.31
4,2,AGCGAACAGCACT,g1|H18735|2010040,21.54,0.31


(28, 5)


In [3]:
folders = os.listdir(motif_dir)
print(folders)
f = 'cluster_H'
print(f)
df = pd.read_csv(os.path.join(motif_dir, f, 'sites.tsv'), sep='\t')
df = df[df.motif_ID!='#']
df['motif'] = [x[2:] for x in df.motif_ID.values]
df['motif_ID'] = [int(x[:1]) for x in df.motif_ID.values]

dg = df.groupby('motif').agg({
    'site_Score':'max'
}).reset_index()

df = df.merge(dg, on=['motif', 'site_Score'], how='inner')
df['site_Start'] = df['site_Start'].astype(int)
df['site_End'] = df['site_End'].astype(int)

df_sites = df[['motif_ID', 'motif', 'site_Start', 'site_End', 'site_Score', 'seq_ID']].drop_duplicates().reset_index(drop=True)
del [df]

display(df_sites.head())
print(df_sites.shape)

['cluster_M_6', 'cluster_H', 'cluster_M_1', 'cluster_M_7', 'cluster_M_2', 'cluster_M_5', 'cluster_L', 'cluster_M_4', 'cluster_M_3']
cluster_H


,motif_ID,motif,site_Start,site_End,site_Score,seq_ID
0,1,AAAACGGTTAGAC,1,13,21.72,g1|H1945|5705031
1,2,AGCGAACAGCACT,9,21,21.54,g1|H18735|4759741
2,2,AGCGAACAGCACT,5,17,21.54,g1|H11251|1008124
3,2,AGCGAACAGCACT,3,15,21.54,g1|H6514|2395052
4,2,AGCGAACAGCACT,3,15,21.54,g1|H18735|2010040


(29, 6)


In [5]:
folders = os.listdir(motif_dir)
print(folders)
f = 'cluster_H'
result = subprocess.run(f"grep -e 'MOTIF' -e 'E=' {os.path.join(motif_dir, f, "streme.txt")}", capture_output=True, text=True, check=True, shell=True)
rv = result.stdout.strip().split()

res = []
tmp = []
for s in rv:
    if s == 'MOTIF':
        if len(tmp) > 0:
            res.append(tmp)
        tmp = []
        continue

    tmp.append(s)

d = []
for r in res:
    d.append({
            'motif_ID': int(r[0][:1]), 
            'motif': r[0][2:],
            'length': int(r[7]),
            'E_value': float(r[11])
            })

df_stream = pd.DataFrame(d)
df_stream

['cluster_M_6', 'cluster_H', 'cluster_M_1', 'cluster_M_7', 'cluster_M_2', 'cluster_M_5', 'cluster_L', 'cluster_M_4', 'cluster_M_3']


,motif_ID,motif,length,E_value
0,1,AAAACGGTTAGAC,13,1.2
1,2,AGCGAACAGCACT,13,3.1
2,3,ACGCAAACGCGA,12,6.2
3,4,AAAGCGGCGTAGGCGT,16,6.9
4,5,AGCAATCGCCGGCG,14,8.1
5,6,CCTCGCGTTTCGA,13,8.1
6,7,CGAAAGAAGACGGC,14,10.0
7,8,CGCCGAACAACGA,13,10.0
8,9,AARACGGTMGSMWKVGSGCGMR,22,10.0


In [7]:
df = df_seq.merge(df_sites, on=['motif_ID', 'motif', 'seq_ID'], how='inner')
df = df.merge(df_stream, on=['motif_ID', 'motif'], how='inner')
display(df.head(30))
print(df.shape)

,motif_ID,motif,seq_ID,seq_Score,motif_P-value,site_Start,site_End,site_Score,length,E_value
0,1,AAAACGGTTAGAC,g1|H1945|5705031,21.72,0.12,1,13,21.72,13,1.2
1,2,AGCGAACAGCACT,g1|H18735|4759741,21.54,0.31,9,21,21.54,13,3.1
2,2,AGCGAACAGCACT,g1|H11251|1008124,21.54,0.31,5,17,21.54,13,3.1
3,2,AGCGAACAGCACT,g1|H6514|2395052,21.54,0.31,3,15,21.54,13,3.1
4,2,AGCGAACAGCACT,g1|H18735|2010040,21.54,0.31,3,15,21.54,13,3.1
5,3,ACGCAAACGCGA,g1|H6244|8185466,18.26,0.62,13,24,18.26,12,6.2
6,3,ACGCAAACGCGA,g1|H21323|6032647,18.26,0.62,10,21,18.26,12,6.2
7,4,AAAGCGGCGTAGGCGT,g1|H6698|7903841,24.60,0.69,10,25,24.60,16,6.9
8,5,AGCAATCGCCGGCG,g1|H11007|5465604,22.68,0.81,5,18,22.68,14,8.1
9,5,AGCAATCGCCGGCG,g1|H13030|4918949,22.68,0.81,6,19,22.68,14,8.1


(26, 10)


In [ ]:
se

In [ ]:
def parse_streme_sequences(file_path):
    """Parses a STREME sequences.tsv file, returning data grouped by motif."""
    data_by_motif = {}
    current_motif_id = None
    
    with open(file_path, 'r', newline='') as tsvfile:
        tsv_reader = csv.DictReader(tsvfile, delimiter='\t')
        
        for row in tsv_reader:
            # Check for new motif block, indicated by "#" line
            if row['motif_ID'].startswith('#'):
                # Handle the separator line
                if 'motif_ID' in row:
                    current_motif_id = row['motif_ID'].lstrip('# ')
                    data_by_motif[current_motif_id] = []
            else:
                if current_motif_id:
                    # Store the relevant information
                    sequence_info = {
                        'sequence_id': row['seq_ID'],
                        'sequence': row['sequence'],
                        'start_position': int(row['start']),
                        'motif_ID': row['motif_ID'],
                        'p-value': float(row['p-value']),
                    }
                    data_by_motif[current_motif_id].append(sequence_info)
    return data_by_motif

# Example usage
file_path = 'sequences.tsv'
streme_data = parse_streme_sequences(file_path)

# Print the extracted information
for motif_id, sequences in streme_data.items():
    print(f"--- Motif: {motif_id} ---")
    for seq in sequences:
        print(f"  Sequence ID: {seq['sequence_id']}, "
              f"Position: {seq['start_position']}, "
              f"Sequence: {seq['sequence']}, "
              f"p-value: {seq['p-value']:.2e}")

In [5]:
folders = os.listdir(motif_dir)
for f in folders:
    print(f)
    fn = os.path.join(motif_dir, f, 'sequences.tsv')
    if os.path.exists(fn):
        print(fn)

cluster_M_6
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_M_6/sequences.tsv
cluster_H
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_H/sequences.tsv
cluster_M_1
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_M_1/sequences.tsv
cluster_M_7
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_M_7/sequences.tsv
cluster_M_2
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_M_2/sequences.tsv
cluster_M_5
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_M_5/sequences.tsv
cluster_L
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_L/sequences.tsv
cluster_M_4
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_M_4/sequences.tsv
cluster_M_3
/Users/mkamruz/Public/SNL/Projects/5UTR/Data/motif_output/more10/STREAM/cluster_M_3/sequences.tsv
